# Quem estiver usando COLAB

In [0]:
from google.colab import drive
# Montar o Google Drive
drive.mount('/content/drive')

In [0]:
# lib de estrutura e análise de dados.
import pandas as pd
# importa database em um arquivo .csv (necessário indicar o diretório)
base = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Aula 4/house_prices.csv')
# exibe os valores
base.head()

# Quem estiver usando ANACONDA

In [0]:
# necessário instalação de pacote pandas.
# lib de estrutura e análise de dados.
import pandas as pd
# importa database em um arquivo .csv (neste caso no mesmo diretório)
base = pd.read_csv('house_prices.csv')
# exibe os valores
base.head()

# INÍCIO TRATAMENTO DE DADOS

In [0]:
# exibe o nome das colunas
base.columns

In [0]:
colunas_usadas = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']

In [0]:
# seleciono apenas as colunas com informações relevantes
colunas_usadas

In [0]:
# ANACONDA
base = pd.read_csv('house_prices.csv', usecols = colunas_usadas)

In [0]:
# COLAB
base = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Aula 4/house_prices.csv', usecols = colunas_usadas)

In [0]:
base.head()

In [0]:
# anteriormente nos usamos o conceito de padronização, onde os dados foram escalonados.
# para esse exemplo vamos usar o conceito de normalização (0 - 1).
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
# inicializamos nosso scaler com os dados, retiramos o PRICE que é nosso objetivo.
base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']] = scaler_x.fit_transform(base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']])
# para cada coluna ele recebe o scaler dos valores de X.

In [0]:
# visualização dos dados normalizados
base.head()

In [0]:
# para o preço que seria considerado nosso Y faremos tb a normalização.
scaler_y = MinMaxScaler()
base[['price']] = scaler_y.fit_transform(base[['price']])

In [0]:
# visualização dos dados normalizados
base.head()

In [0]:
# na variável X nos temos os atributos previsores, logo não queremos os valores do preço nela.
X = base.drop('price', axis = 1) # apaga a coluna do preço.
y = base.price

In [0]:
# visualizar os atributos.
X.head()

In [0]:
# visualizar os atributos.
y.head()

In [0]:
# ajuste dos dados para criação das classes.
previsores_colunas = colunas_usadas[1:17] # posição 0 é o preço
previsores_colunas

# FIM TRATAMENTO DE DADOS.

In [0]:
%tensorflow_version 1.x

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

In [0]:
# vamos criar um for para passar nossas colunas para um array que será usado com o X.
# como os valores de X são numéricos, vamos usar um feature_column.numeric.
colunas = [tf.feature_column.numeric_column(key = c) for c in previsores_colunas]

In [0]:
# visualizar os valores das colunas.
print(colunas[2])

# Separar a base de dados

Vamos separa a base entre treinamento e testes

In [0]:
# agora precisamos criar uma base para testes e outra para o treinamento, normalmente usado 30, 70.
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3)

In [0]:
# visualizar o array X de treino.
X_treinamento.shape

In [0]:
# visualizar o array Y de treino.
y_treinamento.shape

In [0]:
# visualizar o array X de teste.
X_teste.shape

# FUNÇÃO DE TREINAMENTO

In [0]:
# define a função de treinamento.
# passamos as entradas dos valores X e Y que foram separados para testes.
# batch_size é definido com 32 para que o alg não carregue de uma só vez todos os dados.
# 
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento, 
                                                         y = y_treinamento,
                                                         batch_size = 32, 
                                                         num_epochs = None, 
                                                         shuffle = True)

# FUNÇÃO DE TESTE

In [0]:
# define a função de teste.
funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_teste, 
                                                   y = y_teste,
                                                   batch_size = 32, 
                                                   num_epochs = 10000, 
                                                   shuffle = False)

# ALGORITMO - REGRESSÃO LINEAR

In [0]:
# função do regressor.
regressor = tf.estimator.LinearRegressor(feature_columns=colunas)

TREINAMENTO...

In [0]:
# com o regressor podemos fazer o treinamento.
regressor.train(input_fn=funcao_treinamento, steps = 10000)

In [0]:
# calculo das metricas de treinamento.
metricas_treinamento = regressor.evaluate(input_fn=funcao_treinamento, steps = 10000)

TESTES...

In [0]:
# calculo das metricas de testes. (função de avaliação)
metricas_teste = regressor.evaluate(input_fn=funcao_teste, steps = 10000)

In [0]:
# exibindo valores da métrica.
metricas_treinamento

In [0]:
# exibindo valores da métrica. (valores para análise de resultados.)
metricas_teste

PREVISÃO...

In [0]:
# função de previsão, gera os valores para cada um dos valores de X.
funcao_previsao = tf.estimator.inputs.pandas_input_fn(x = X_teste, shuffle = False)

In [0]:
previsoes = regressor.predict(input_fn=funcao_previsao)

In [0]:
# exibe valores previstos.
list(previsoes)

In [0]:
# para o calculo do erro, vamos criar um array, que irá receber cada um dos valores gerados pela função previsão.
valores_previsoes = []
for p in regressor.predict(input_fn=funcao_previsao):
    valores_previsoes.append(p['predictions'])

In [0]:
# exibe array com os valores.
# com esse valores iremos calcular os erros MSE e MAE.
valores_previsoes

In [0]:
# como os valores foram normalizados, precisamos desnormaliza-los.
import numpy as np
# colocar no formato de matriz.
valores_previsoes = np.asarray(valores_previsoes).reshape(-1,1)

In [0]:
valores_previsoes

In [0]:
valores_previsoes = scaler_y.inverse_transform(valores_previsoes)
valores_previsoes

In [0]:
y_teste

In [0]:
y_teste.shape

In [0]:
y_teste2 = y_teste.values.reshape(-1,1)
y_teste2.shape

In [0]:
y_teste2 = scaler_y.inverse_transform(y_teste2)

In [0]:
y_teste2

In [0]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_teste2, valores_previsoes)

In [0]:
mae